In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import plotly.graph_objects as go
import pickle
import sys
import os
from scipy.signal import butter,filtfilt # Filter requirements.
from scipy.spatial.transform import Rotation as R

os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

import opensim as osim

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from util.Object_util import getHandVectors, visualizeHandVectors, createObjectID, createObjectTraj, createExternalLoadsData
from OsimPython.osim_file_util import readFile_trc, createFile_trc, createFile_mot, readFile_mot, readFile_ID_sto
from util.plotUtil import addArrow, addMarker

data_path = 'D:\Dokumente\Projects\OOA\Data\\' # root of all data
path_MoCap = 'MoCap\\' # where the trial .trc data lives
object_path = data_path + 'Objects\\' # where object data lives

' TRIAL SETTINGS'
obj_name = 'Can'
experiment_name = 'example_trial'

# Create an Object Model in OpenSim
Here object models and object settings are created which will be used for: <br>
1. placing object markers in hand coordinate frames
2. performing inverse kinematics to derive object motion from object markers
3. compute object inverse dynamics from object motionm


In [ ]:
name = "Can"
r = 0.074/2 # radius
h = 0.109 # height
m = 0.5077 # mass kg

# Create empty model and assign the name "leaver" to it.
model = osim.Model()
model.setName(name)

# If inverse dynamics are done in another program this might come in handy
#model.setGravity(osim.Vec3(0,0,-9.8066499999999994))

# Define body
cylinder = osim.Body("cylinder", # name
                    m, # mass kg
                    osim.Vec3(0), # mass center (0 means its at the center of the thing)
                    osim.Inertia(1/12*m*(3*r**2+h**2), 1/12*m*(3*r**2+h**2), 1/2*m*r**2)) # intertia, cylinder aligned with z axis

# Define joint 
basejoint = osim.FreeJoint("CoMjoint", #rotates around common Z axis of joint frames
                         model.getGround(), # PhysicalFrame
                         osim.Vec3(0, 0, 0), # location in parent frame
                         osim.Vec3(0, 0, 0), # orientation in parent frame # we rotate around x because opensim has the y axis up and in all my scripts the z axis is up
                         cylinder, # PhysicalFrame
                         osim.Vec3(0, 0, 0),  # attach the joint in the center of the object
                         osim.Vec3(0)) 

# Define virtual OpenSim Markers for inverse kinematics
# The location of these markers must match with the markers which were measured or inferred from motion capture
CoMMarker = osim.Marker('CoM', # name
                        cylinder, # PhysicalFrame
                        osim.Vec3(0, 0, 0) # location in cylinder frame               
                        )

TipMarker = osim.Marker('Tip', # name
                        cylinder, # PhysicalFrame
                        osim.Vec3(0, 0, h/2) # location in cylinder frame               
                        )

HullMarker = osim.Marker('Hull', # name
                        cylinder, # PhysicalFrame
                        osim.Vec3(0, r, 0) # location in cylinder frame               
                        )

# Add all of the elements defined in previous cells to the model.
model.addBody(cylinder)
model.addJoint(basejoint)
model.addMarker(CoMMarker)
model.addMarker(TipMarker)
model.addMarker(HullMarker)

# Add display geometry. 
'''careful, the geometry is wrong! it is rotated around x by 90°''' 
bodyGeometry = osim.Cylinder(r, h/2) # radius and half height
bodyGeometry.setColor(osim.Red)
cylinderCenter = osim.PhysicalOffsetFrame()
cylinderCenter.setName("CylinderCenter")
cylinderCenter.setParentFrame(cylinder)

R = osim.Rotation()
# R.setRotationFromAngleAboutX(np.pi/2)

cylinderCenter.setOffsetTransform(osim.Transform(R, osim.Vec3(0, 0, 0)))
cylinder.addComponent(cylinderCenter)
cylinderCenter.attachGeometry(bodyGeometry.clone())

# save model
model.finalizeConnections()
model.printToXML(object_path + name + '.osim')

# This dictionary contains the dimensions, mass, name and marker information of this object
# Marker information contains marker names and their offsets in (n, p, f) of hand corrdinate frames in meters
object_dict = dict(r = r, 
                   h = h, 
                   m = m, 
                   name = name,
                   markerNames = ['CoM', 'Tip', 'Hull'],
                   markers = [(0.047+r, -0.008, 0.028), (0.047+r, h/2-0.008, 0.028), (0.047, -0.008, 0.028)]) # location of object markers in hand coordiate frames. each tuple contains (offset in directioon of palm normal, offset in direction of upwards vector based on ulnar to radial styloid, and the offset in the finger forward direction)

with open(object_path + name + '.pkl',"wb") as filehandler:
    pickle.dump(object_dict, filehandler)

# Prepare MoCap Data

Here the MoCap data is prepared. <br>
Most importantly the orientation needs to match with the OpenSim coordinate frame (y-axis up).  <br>
This is important for the correct direction of gravity. <br>
If that is not the case a simple rotation matrix can be used with the functionalities in the OsimPython folder. <br>

In [ ]:
# check marker naming
markerNames,  marker_xyz, timeseries, frameseries, DataRate, data_dict_MoCap = readFile_trc(data_path + path_MoCap, experiment_name + '.trc')
fig = go.Figure()
frame = 1000 # input any frame to check marker labels.

# in the example ground truth object markers are included (Can1 - Can3)
for i, mn in enumerate(markerNames):
    fig = addMarker(fig, marker_xyz[frame, i, :], name=mn)
fig.show()

# Generate Object Traj and wrench

1. Hand coordinate frames are computed from hand and limb markers as described in <br>
Herneth et al. (2024) Object Augmentation Algorithm: Computing virtual object motion and object induced interaction wrench from optical markers <br>
<br>
2. Objects are placed in hand frames by their dimensions or other transformations <br>
<br>
3. Object induced interaction wrench is computed via OpenSim <br>
 

Get hand frames at each marker frame

In [ ]:
# General Settings
hmnR = ['Hand3', 'Hand4', 'Hand5', 'Hand6', 'Hand7'] # palm markers R
wmnR = ['ForearmUL', 'ForearmRS'] # from ulnar to radial styloid 
famnR = ['ElbowLE', 'ForearmUL'] # from ellbow to wrist R (to calculate the correct direction of the palm normal vector)
experiment_hand = 'R'

with open(data_path + path_MoCap + experiment_name + '_wrist_deviation_angle.pkl',"rb") as input_file:
    matlab_trial_dict = pickle.load(input_file)

# load prepared MoCap data
markerNames,  marker_xyz, timeseries, frameseries, DataRate, data_dict_MoCap = readFile_trc(data_path + path_MoCap, experiment_name + '.trc')

dev_ang_R = matlab_trial_dict['qJ'][:, 3] # already in radians

experiment_dict = dict(
    experiment_name = experiment_name,
    d = marker_xyz, 
    timeseries = timeseries, 
    data_dict_MoCap = data_dict_MoCap,
    # MoCap derived data
    HandMarkersR = marker_xyz[:, [markerNames.index(mn) for mn in hmnR], :],
    WristMarkersR = marker_xyz[:, [markerNames.index(mn) for mn in wmnR], :],
    ForearmMarkersR = marker_xyz[:, [markerNames.index(mn) for mn in famnR], :],
    dev_ang_R = dev_ang_R, # numpy wants radians
)

# Calculate series of vec2 (rotated styloid vector), vec1 (palm normal) 
# from previosly selected markers and vec3 = cross product vec1 x vec2 (points in the direction of the fingers)
# These vectors will be used to calculate the object markers
data = getHandVectors(experiment_dict, hand='R') # This is the actual OAA implementation
centroidsR, nR, pR, fR = data[0], data[1], data[2], data[3]

hand_dict = dict(centroidsR = centroidsR,
                 nR = nR, 
                 pR = pR, 
                 fR = fR, 
                 )

# saves the hand vector series
with open(data_path + 'Hand_frames/' + experiment_name + '_handFrames.pkl',"wb") as filehandler:
    pickle.dump(hand_dict, filehandler)

# visualizes handvectors along the given dataslice
fig = visualizeHandVectors(experiment_dict, hand_dict, experiment_hand, d_slice=(0, 1))

# Plots a coordinate frame at the 0 location
fig = addArrow(fig, [0, 0, 0], np.array([1, 0, 0])*100, color='red')
fig = addArrow(fig, [0, 0, 0], np.array([0, 1, 0])*100, color='green')
fig = addArrow(fig, [0, 0, 0], np.array([0, 0, 1])*100, color='blue')

fig.show()

Initialize Object

In [ ]:
# here the object trajectory is calculated based on the previosly computed hand coordinate frames and the positions of object markers in hand coordinateframes
# the objects are generated in createObjectModels.ipynb, where also the object marker location in hand frames is defined

object_data_filename = experiment_dict['experiment_name'] + '_' + obj_name

obj_dict = dict(obj_name = obj_name, 
                obj_path = object_path, 
                filename = object_data_filename
                )

Create Object IK

In [ ]:
rad = 0 # fixed object rotation around the hand frames p-axis in degrees
c_offset = 0 # fixed object offset in hand frame f direction in mm

# places objects in each hand corrdinate frame and then uses opensim to compute inverse kinematics.
# If the markers errors are large (larger than 100um) then the virtual OpenSim markers and the definition of marker placements within hand corrdinate frames does not match.
# Have a look here at: # Define virtual OpenSim Markers for inverse kinematics and object_dict in createObjectModels.ipynb
createObjectTraj(experiment_dict, obj_dict, hand_dict, experiment_hand, Units=experiment_dict['data_dict_MoCap']['Units'], 
                      hand_vec='nR', rad=rad*np.pi/180, c_offset=c_offset, verbose=1)


# filter object traj (remove high frequency noise caused by marker wobble from MoCap)
coordList, coord_val, timeseries, inDegrees, data_dict = readFile_mot(obj_dict['obj_path'], object_data_filename+'.mot')
b, a = butter(N=3, Wn=3, btype='low', analog=False, fs=100) # N = order, Wn = cutoff
coord_val_filt = filtfilt(b, a, coord_val, axis=0)


fig = go.Figure()
print('Have a look and adjust the filtering accordingly')
# unfiltered
for i in range(len(coordList)):
    fig.add_trace(go.Scatter(x=timeseries, y=coord_val[:, i],
                        mode='lines',
                        name=coordList[i], 
                        ))
# filtered   
for i in range(len(coordList)):
    fig.add_trace(go.Scatter(x=timeseries, y=coord_val_filt[:, i],
                        mode='lines',
                        name=coordList[i]+'_filt', 
                        line=dict(dash='dot')))

fig.show()

# save thr filtered time series of object markers computed from limb, wrist and hand markers
createFile_mot(obj_dict['obj_path'], object_data_filename+'.mot', model=osim.Model(object_path+obj_name+'.osim'), coord_val=coord_val_filt, time=timeseries, Unit='deg')

Object inverse dynamics

Uses the OpenSim inverse dynamics tool to compute the wrench necessary to move objects along the prescribed trajectory 

In [ ]:
obj_dict['filename_ending'] = '' # if we dont use createCutRotObjectTraj

createObjectID(obj_dict, timeseries, lowpass_cutoff_frequency_for_coordinates=-1, verbose=0) # makes the ID of the objects and saves it

# Now we load the inverse dynamics date just generated and display it
# In the example we see on major force compoinent in y direction (5N)
# This matched with the expectation of a relatively slow motion, where only gravity is dominant (the can has 500g) and OpenSim gravity is in -y direction
coord_val_names, coord_val_tau, _, _, _ = readFile_ID_sto(path=obj_dict['obj_path'], filename=object_data_filename + '' +'.sto')
_, coord_val_loc, _, _, _ = readFile_mot(obj_dict['obj_path'], object_data_filename+'.mot')

fig = go.Figure()
for i in range(6):
    # Plot the knee angles on the first subplot.
    fig.add_trace(go.Scatter(x=timeseries, y=coord_val_tau[:, i],
                mode='lines',
                name=coord_val_names[i], 
                line=dict(dash='solid')
                ))
fig.show()

finished! <br>
The Object Trajectory is now available togehter with the wrenvh necessary to move the object along the prescribed trajectory. <br>
If this wrench is to be used to emulate object handling in an inverse dynamics simulation of a person handling the same object an external loads setup can be used

In [ ]:
createExternalLoadsData(obj_dict, 'Hand')

# The following is a comparison between the computed object markers and the ground truth markers recorded in the example

Grounth Truth Object Marker Trajectories

In [ ]:
# units are in mm
# First a new object is created for the ground truth data
obj_path = data_path + 'Objects/'
object_data_filename_GM = experiment_dict['experiment_name'] + '_' + obj_name + '_vicon'

obj_dict_GM = dict(obj_name = obj_name, 
                obj_path = obj_path, 
                filename = object_data_filename_GM,
                filename_ending = ''
                )

From the ground truth markers and the OpenSim object above a new object trajectory is calculated

In [ ]:
# Create VIRTUAL MARKER Trajectories
with open(object_path + obj_name + '.pkl', "rb") as input_file:
    obj_dict = pickle.load(input_file)

# Ground truth Trajectories
Can_viconM = marker_xyz[:, [markerNames.index(mn) for mn in ['Can1', 'Can2', 'Can3']], :] # ground truth markers
# for trial 21 markers 0 and 1 are swapped

# Here the ground truth marker locations are tranformed to virtual gorund truth marker locations that corespond to those 
# produced by the algorithm. This is because no optivcal markers can be placed at the objects CoM. Instead markers were placed on a known shape and are
# here transformed to the Tip, CoM and Hull 
Can_GM = np.zeros([Can_viconM.shape[0], 3, 3]) # 3 markers, 3 coordinates
n_vecs = np.zeros([Can_viconM.shape[0], 3])
for i in range(Can_viconM.shape[0]):
    v1 = Can_viconM[i, 2, :] - Can_viconM[i, 1, :] # vector from marker 1 to marker 2
    v1 /= np.linalg.norm(v1) # vector from marker to to marker 2
    v2 = Can_viconM[i, 0, :] - Can_viconM[i, 1, :] # vector from marker 1 to marker 0
    v2 /= np.linalg.norm(v2) 
    n = np.cross(v1, v2) # vector across vertical axis of can
    n /= np.linalg.norm(n)
    n_vecs[i] = n

    if obj_name == 'Can':
        h = obj_dict['h']
        r = obj_dict['r']
        c = Can_viconM[i, 1, :] + v1 * 30.62 - v2 * 9.15 # center of the circle, where the markers lie on (measurements from solidworks)
        Can_GM1 = c - (9+h/2*1000) * n # CoM Marker
        Can_GM2 = c - 9 * n # Tip Marker
        Can_GM3 = c - (9+h/2*1000) * n + v2 * r*1000 # Hull Marker
        Can_GM[i, 0, :] = Can_GM1
        Can_GM[i, 1, :] = Can_GM2
        Can_GM[i, 2, :] = Can_GM3

# fiel containing virtual ground truth object markers
createFile_trc(obj_path, object_data_filename_GM+'.trc', Can_GM, ['CoM', 'Tip', 'Hull'], DataRate=100, Unit='mm')

MarkerNames_GM,  marker_xyz_obj_GM, _, _, _, _ = readFile_trc(obj_path, object_data_filename_GM+'.trc') # ground truth
MarkerNames,  marker_xyz_obj, _, _, _, _ = readFile_trc(obj_path, object_data_filename+'.trc') # algorithm output

print('Make sure the marker order is the same!!!')
print(MarkerNames_GM, MarkerNames)

Compare x,y,z locations in the hand frame 

In [ ]:
i = 0
fig = visualizeHandVectors(experiment_dict, hand_dict, experiment_hand, d_slice=(i, i+1))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj_GM[i, 0, 0]],
            y=[marker_xyz_obj_GM[i, 0, 1]],
            z=[marker_xyz_obj_GM[i, 0, 2]],
            mode='markers+text',
            text='CoM_VM',
            marker=dict(
                size=5,
                color='red',
            )
    ))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj_GM[i, 1, 0]],
            y=[marker_xyz_obj_GM[i, 1, 1]],
            z=[marker_xyz_obj_GM[i, 1, 2]],
            mode='markers+text',
            text='Tip_VM',
            marker=dict(
                size=5,
                color='blue',
            )
    ))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj_GM[i, 2, 0]],
            y=[marker_xyz_obj_GM[i, 2, 1]],
            z=[marker_xyz_obj_GM[i, 2, 2]],
            mode='markers+text',
            text='Hull_VM',
            marker=dict(
                size=5,
                color='green',
            )
    ))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj[i, 0, 0]],
            y=[marker_xyz_obj[i, 0, 1]],
            z=[marker_xyz_obj[i, 0, 2]],
            mode='markers+text',
            text='CoM_AM',
            marker=dict(
                size=5,
                color='orange',
            )
    ))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj[i, 1, 0]],
            y=[marker_xyz_obj[i, 1, 1]],
            z=[marker_xyz_obj[i, 1, 2]],
            mode='markers+text',
            text='Tip_AM',
            marker=dict(
                size=5,
                color='cyan',
            )
    ))

fig.add_trace(go.Scatter3d(
            x=[marker_xyz_obj[i, 2, 0]],
            y=[marker_xyz_obj[i, 2, 1]],
            z=[marker_xyz_obj[i, 2, 2]],
            mode='markers+text',
            text='Hull_AM',
            marker=dict(
                size=5,
                color='yellow',
            )
    ))

print('The Tip and CoM markers match well with the ground truth')
print('The hull marker is rotated, since no rotation attention was paid to its placement in the hand frame compared to the round truth.')
print('For rotation symmetric objects this does not influence inverse dynamics, while for other objects the location of that marker must be carefully chosen when creating the Osim object')

fig.show()

Compare xyz marker trajectories

In [ ]:
# compare xyz
fig = go.Figure()
b, a = butter(N=3, Wn=10, btype='low', analog=False, fs=100)
#for i in range(marker_xyz_obj.shape[0]):
print(MarkerNames[0])
print('Marker trajectories match very well between grounth truth and those prodeuced by the algorithm')
fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 0, 0],
            mode='lines',
            name='x_VM',
            marker=dict(
                size=1,
                color='red',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 0, 0], axis=0),
            mode='lines',
            name='x_A',
            marker=dict(
                size=1,
                color='orange',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 0, 1],
            mode='lines',
            name='y_VM',
            marker=dict(
                size=1,
                color='red',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 0, 1], axis=0),
            mode='lines',
            name='y_A',
            marker=dict(
                size=1,
                color='orange',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 0, 2],
            mode='lines',
            name='z_VM',
            marker=dict(
                size=1,
                color='red',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 0, 2], axis=0),
            mode='lines',
            name='z_A',
            marker=dict(
                size=1,
                color='orange',
            )
    ))

fig.show()

fig = go.Figure()
print(MarkerNames[1])
#for i in range(marker_xyz_obj.shape[0]):
fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 1, 0],
            mode='lines',
            name='x_VM',
            marker=dict(
                size=1,
                color='blue',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 1, 0], axis=0),
            mode='lines',
            name='x_A',
            marker=dict(
                size=1,
                color='green',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 1, 1],
            mode='lines',
            name='y_VM',
            marker=dict(
                size=1,
                color='blue',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 1, 1], axis=0),
            mode='lines',
            name='y_A',
            marker=dict(
                size=1,
                color='green',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=marker_xyz_obj_GM[:, 1, 2],
            mode='lines',
            name='z_VM',
            marker=dict(
                size=1,
                color='blue',
            )
    ))

fig.add_trace(go.Scatter(
            x=timeseries,
            y=filtfilt(b, a, marker_xyz_obj[:, 1, 2], axis=0),
            mode='lines',
            name='z_A',
            marker=dict(
                size=1,
                color='green',
            )
    ))

fig.show()

In [ ]:
import numpy as np

def calc_correlation(actual, predic):
    a_diff = actual - np.mean(actual)
    p_diff = predic - np.mean(predic)
    numerator = np.sum(a_diff * p_diff)
    denominator = np.sqrt(np.sum(a_diff ** 2)) * np.sqrt(np.sum(p_diff ** 2))
    return numerator / denominator

print('corr x M1')
res0 = calc_correlation(marker_xyz_obj_GM[:, 0, 0], marker_xyz_obj[:, 0, 0])
print(res0)

print('corr x M2')
res1 = calc_correlation(marker_xyz_obj_GM[:, 1, 0], marker_xyz_obj[:, 1, 0])
print(res1)

print()
print('corr y M1')
res0 = calc_correlation(marker_xyz_obj_GM[:, 0, 1], marker_xyz_obj[:, 0, 1])
print(res0)

print('corr y M2')
res1 = calc_correlation(marker_xyz_obj_GM[:, 1, 1], marker_xyz_obj[:, 1, 1])
print(res1)

print()
print('corr z M1')
res0 = calc_correlation(marker_xyz_obj_GM[:, 0, 2], marker_xyz_obj[:, 0, 2])
print(res0)

print('corr z M2')
res1 = calc_correlation(marker_xyz_obj_GM[:, 1, 2], marker_xyz_obj[:, 1, 2])
print(res1)
print()
print('The correlation is very high')